# Beispiel digitale Modulation

Dieses [Jupyter Notebook](https://de.wikipedia.org/wiki/Project_Jupyter#Jupyter_Notebook) gibt einen ersten Einblick in die Thematik der digitalen [Modulation](https://de.wikipedia.org/wiki/Modulation_(Technik)) und dient als interaktives Begleitmaterial zu dem Modul "Einführung in die IKT" des Studiengangs [IKT](https://ikt-bachelor.htw-berlin.de/) der [HTW Berlin](https://www.htw-berlin.de/).

Zusätzlich kann es auch als ein erster Kontakt mit der Programmiersprache Python gesehen werden, da der Code des interaktiven Notebooks eingesehen, angepasst und erweitert werden kann.

Um dieses Notebook auf Ihrem Rechner ausführen zu können benötigen Sie eine Python Installtion und zusätzlich die Module
* [numpy](https://numpy.org/)
* [matplotlib](https://matplotlib.org/)
* [jupyter](https://jupyter.org/) und
* [jupyter-widgets](https://ipywidgets.readthedocs.io/en/stable/).

Bei Rückfragen oder Problemen wenden Sie sich einfach gerne an [mich](https://www.htw-berlin.de/hochschule/personen/person/?eid=9586).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import ipywidgets as widgets

plt.style.use('noelle.mplstyle')

In [2]:
def bits_to_dec(bits, m):
    """ Convert 1D array of bits into 1D array of decimals, using a resolution of m bits.
    
    """
    bits = np.asarray(bits)    
    
    bits_reshape = bits.reshape((-1, m))
    bit_val = np.reshape(2**(np.arange(m-1,-1,-1)),(1,-1))
    decimals = np.sum(bits_reshape * bit_val, axis=-1).astype(int)
    
    return decimals

In [3]:
def dec_to_bits(decimals, m):
    """ Convert 1D array of decimals into 1D array of bits, using a resolution of m bits.
    
    """
    decimals = np.asarray(decimals)
    
    if decimals.ndim > 1:
        raise ValueError('dimension of bits should be <=1...')        
    
    bits = np.full((decimals.size,m), np.nan)
    tmp = decimals
    
    for bit in np.arange(m-1,-1,-1):        
        bits[:, bit] = tmp % 2
        tmp = tmp // 2
                        
    bits = bits.reshape(decimals.size*m).astype(bool)
    
    return bits

In [4]:
def mapper(bits, constellation):
    """ Map bits to a given constellation alphabet. 
    
    """
    m = int(np.log2(constellation.size))
    decimals = np.full((int(bits.shape[0]/m),), np.nan)
    
    if m == 1:
        decimals = bits        
    else:
        decimals = bits_to_dec(bits, m)
    
    symbols = constellation[decimals.astype(int)]
    
    return symbols

In [5]:
def demapper(samples, constellation):
    """    
    Demap samples to bits using a given constellation alphabet.   

    """
    samples = np.asarray(samples)
    constellation = np.asarray(constellation)

    if constellation.ndim > 1:
        raise ValueError('number of dimensions of constellation must not exceed 1!')

    if samples.ndim > 1:
        raise ValueError('number of dimensions of samples must not exceed 1!')

    decimals = np.full_like(samples.real, np.nan)
    bps = int(np.log2(constellation.size))    
    
    for const_idx, const_point in enumerate(constellation):
        decimals[samples == const_point] = const_idx
    
    # convert constellation index (decimal) to bits
    bits = dec_to_bits(decimals, bps)        
  
    return bits

In [6]:
def generate_and_plot_signals(mod_format='OOK'):
    """ Generate constellation, baseband, carrier and bandpass (modulated) signal and plot them.
    
    """
    
    # some parameters (can be changed)
    n_symbols = 6
    sample_rate = 100
    symbol_rate = 1 # fix!
    carrier_freq = 2    
    # time axis
    t = np.arange(n_symbols*sample_rate) / sample_rate
    # carrier signal
    carrier = np.exp(2j * np.pi * carrier_freq * t)
    # set constellation according modulation format
    if mod_format == 'OOK':
        const = np.asarray([0, 1])
        const = const / np.max(np.abs(const))
    elif mod_format == 'BPSK':
        const = np.asarray([-1, 1])
        const = const / np.max(np.abs(const))
    elif mod_format == '4-ASK (unipolar)':
        const = np.asarray([0, 1, 2, 3])
        const = const / np.max(np.abs(const))
    elif mod_format == '4-ASK (bipolar)':
        const = np.asarray([-3, -1, 3, 1])
        const = const / np.max(np.abs(const))
    elif mod_format == 'QPSK':
        const = np.asarray([-1-1j, -1+1j, 1-1j, 1+1j])
        const = const / np.max(np.abs(const))
    elif mod_format == '16-QAM':
        const = np.asarray([-3-3j, -3-1j, -3+1j, -3+3j, -1-3j, -1-1j, -1+1j, -1+3j, 1-3j, 1-1j, 1+1j, 1+3j, 3-3j, 3-1j, 3+1j, 3+3j])
        const = const / np.max(np.abs(const))
    else:
        raise ValueError('Modulation format not known')
    
    bits_per_symbol = int(np.log2(const.size))
    # generate random bits
    bits = np.random.randint(0, high=2, size=n_symbols*bits_per_symbol)
    # map bits to symbols according constellation
    symbols = mapper(bits, const)
    
    # plot the signals
    n_row = 1
    n_col = 3
    fig_size = [i*j for i,j in zip(plt.rcParams['figure.figsize'], [n_col, n_row])]
    fig = plt.figure(figsize=fig_size)
    # unmodulated carrier
    ax = fig.add_subplot(n_row, n_col, 1)
    ax.plot(t, np.real(carrier))
    ax.set_ylim((-np.max(np.abs(const))-1, np.max(np.abs(const))+1))
    ax.set(xlabel="time / s", ylabel="amplitude / a.u.",title='unmodulated carrier')
    #  constellation
    ax = fig.add_subplot(n_row, n_col, 2)
    ax.plot(const.real, const.imag, 'o')
    for idx, symbol in enumerate(const):
        ax.annotate(str(dec_to_bits(idx,m = int(np.log2(const.size))).astype(int)),(symbol.real-0.1, symbol.imag+0.1), xycoords='data', size=8)    
    ax.set_ylim((np.min(const.imag)-1, np.max(const.imag)+1))
    ax.set_xlim((np.min(const.real)-1, np.max(const.real)+1))
    ax.set(xlabel="real part", ylabel="imaginary part",title='constellation diagram')
    ax.set_aspect('equal', 'box')
    # modulated signal
    ax = fig.add_subplot(n_row, n_col, 3)
    ax.plot(t, np.real(np.repeat(symbols, int(sample_rate/symbol_rate)) * carrier))
    for idx, symbol in enumerate(symbols):
        ax.annotate(str(demapper(symbol, const).astype(int)),(idx, np.max(np.abs(symbols))+0.3), xycoords='data', size=8)
    ax.set_ylim((-np.max(np.abs(const))-1, np.max(np.abs(const))+1))
    ax.set(xlabel="time / s", ylabel="amplitude / a.u.",title='modulated carrier')
    fig.set_tight_layout(True)
    plt.show()

In [7]:
w_mod_format = widgets.RadioButtons(options=['OOK', 'BPSK', '4-ASK (unipolar)', '4-ASK (bipolar)', 'QPSK', '16-QAM'], value='OOK', description='Modulation format:', disabled=False)

out = widgets.interactive_output(generate_and_plot_signals, {'mod_format':w_mod_format})
out.layout.height = '300px'
# display all
display(w_mod_format, out)

RadioButtons(description='Modulation format:', options=('OOK', 'BPSK', '4-ASK (unipolar)', '4-ASK (bipolar)', …

Output(layout=Layout(height='300px'))